This file's objective is to prototype an internal representation of the data for ease of use.

In [1]:
import polars as pl

DATA_DIR = "../data/"
FILE_PATH = "alpha_investing_new.csv"

In [2]:
df = pl.read_csv(DATA_DIR + FILE_PATH)
df

algorithm,n,g_in,h,G_out,coverage,diversity,power,fdr,time
str,i64,str,str,str,f64,f64,f64,f64,f64
"""covdiv_alpha""",1,"""gender:F""","""Mean is greater than 3.9""","""gender:F|year:50s""",0.681685,0.0,1.0,0.0,1.814997
"""covdiv_alpha""",2,"""gender:F""","""Mean is greater than 3.9""","""gender:F|year:50s, gender:F|ru…",0.725571,0.695968,1.0,0.0,2.036166
"""covdiv_alpha""",3,"""gender:F""","""Mean is greater than 3.9""","""gender:F|runtime_minutes:Short…",0.761849,1.983853,1.0,0.0,1.968691
"""covdiv_alpha""",4,"""gender:F""","""Mean is greater than 3.9""","""gender:F|runtime_minutes:Short…",0.854886,3.694056,1.0,0.0,2.120133
"""covdiv_alpha""",5,"""gender:F""","""Mean is greater than 3.9""","""gender:F|year:30s, gender:F|ru…",0.873025,5.645222,1.0,0.0,2.093583
…,…,…,…,…,…,…,…,…,…
"""cover_alpha""",16,"""gender:F""","""Mean is greater than 3.9""","""gender:F|year:50s, gender:F|ru…",0.89175,7.9206,1.0,0.0,2.142911
"""cover_alpha""",17,"""gender:F""","""Mean is greater than 3.9""","""gender:F|runtime_minutes:Short…",0.89175,7.9206,1.0,0.0,2.068365
"""cover_alpha""",18,"""gender:F""","""Mean is greater than 3.9""","""gender:F|runtime_minutes:Very …",0.89175,7.9206,1.0,0.0,2.158641


In [3]:
type(df[15])

polars.dataframe.frame.DataFrame

In [16]:
g_out = df["G_out"][15].split(",")
def groups_from_row(g_out: str) -> dict[str, str]:
    new_groups = []
    for g in g_out:
        chars = g.split("|")
        char_dict = {}
        for c in chars:
            k, v = c.split(":")
            char_dict[k] = v
        new_groups.append(char_dict)
    return new_groups
groups_from_row(g_out)


[{'gender': 'F', 'year': '50s'},
 {' gender': 'F', 'runtime_minutes': 'Very Long'},
 {' gender': 'F', 'genre': 'Film-Noir'},
 {' gender': 'F', 'year': '40s'},
 {' gender': 'F', 'year': '30s'},
 {' gender': 'F', 'runtime_minutes': 'Short'}]

In [18]:
class HypothesisRow:
    algorithm: str
    n: int
    parent: dict[str, str]
    hypothesis: str
    children: list[dict[str, str]]
    coverage: float
    diversity: float
    power: float
    fdr: float
    time: float


    def __init__(self, row: tuple):
        self.algorithm = row[0]
        self.n = row[1]
        self.parent = HypothesisRow.group_from_str(row[2])
        self.hypothesis = row[3]
        self.children = HypothesisRow.groups_from_row(row[4])
        self.coverage = row[5]
        self.diversity = row[6]
        self.power = row[7]
        self.fdr = row[8]
        self.time = row[9]


    @staticmethod
    def group_from_str(s: str) -> dict[str, str]:
        attributes = s.split("|")
        attr_dict = {}
        for a in attributes:
            try:
                k, v = a.split(":")
                attr_dict[k] = v
            except:
                print(a)
                raise ValueError
        return attr_dict
    

    @staticmethod
    def groups_from_row(s: str) -> list[dict[str, str]]:
        groups = s.split(",")
        return [HypothesisRow.group_from_str(g) for g in groups]


In [4]:
df[15]

algorithm,n,g_in,h,G_out,coverage,diversity,power,fdr,time
str,i64,str,str,str,f64,f64,f64,f64,f64
"""covdiv_alpha""",16,"""gender:F""","""Mean is greater than 3.9""","""gender:F|year:50s, gender:F|ru…",0.89175,7.9206,1.0,0.0,1.931885


In [5]:
df["G_out"][15]

'gender:F|year:50s, gender:F|runtime_minutes:Very Long, gender:F|genre:Film-Noir, gender:F|year:40s, gender:F|year:30s, gender:F|runtime_minutes:Short'

In [9]:
%run ../hypothesis_row.py

i = 0
for r in df.iter_rows():
    if i < 15:
        i += 1
        continue
    ex = HypothesisRow(r)
    break
vars(ex)

{'algorithm': 'covdiv_alpha',
 'n': 16,
 'parent': {'gender': 'F'},
 'hypothesis': 'Mean is greater than 3.9',
 'children': [{'gender': 'F', 'year': '50s'},
  {' gender': 'F', 'runtime_minutes': 'Very Long'},
  {' gender': 'F', 'genre': 'Film-Noir'},
  {' gender': 'F', 'year': '40s'},
  {' gender': 'F', 'year': '30s'},
  {' gender': 'F', 'runtime_minutes': 'Short'}],
 'coverage': 0.8917495611468695,
 'diversity': 7.920599738565601,
 'power': 1.0,
 'fdr': 0.0,
 'time': 1.931885004043579}